<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Проект-для-«Викишоп»" data-toc-modified-id="Проект-для-«Викишоп»-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Проект для «Викишоп»</a></span><ul class="toc-item"><li><span><a href="#Модель-обучения-без-BERT" data-toc-modified-id="Модель-обучения-без-BERT-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Модель обучения без BERT</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Модель-обучения-с-BERT" data-toc-modified-id="Модель-обучения-с-BERT-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Модель обучения с BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Модель обучения без BERT

## Подготовка

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer #,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import torch
import transformers

import random as rnd

In [2]:
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')

Подгружаем данные:

In [3]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные загружены. Имеем ~160 тыс. размеченных комментариев. Начнем с грубого анализа состояния данных. Смотрим наличие пропусков:

In [6]:
data.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков нет. Смотрим на баланс классов:

In [7]:
data.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Имеем **сильный дисбаланс** двух классов в соотношении $\frac19$, где 1 - класс токсичных комментариев.

Посмотрим на несколько комментариев из каждой категории:

In [8]:
data.loc[data.toxic == 1]['text'].head(3) 

6          COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
12    Hey... what is it..\n@ | talk .\nWhat is it......
16    Bye! \n\nDon't look, come or think of comming ...
Name: text, dtype: object

In [9]:
data.loc[data.toxic == 0]['text'].tail(3)

159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: text, dtype: object

В комментариях присутствуют знаки переноса, пунктуации и @. От всего этого нам предстоит избавиться.

Для начала, произведем токенизацию и лемматизацию комментариев:

In [10]:
def clean_text(expression, replacement, text):
    text_wo_re = re.sub(expression, replacement, text)
    return ' '.join(text_wo_re.split())

In [11]:
def tokenize(text):
    return nltk.word_tokenize(text)

In [12]:
def lemmatize(text, expression):
    # initializing a lemmatizer
    wnl = WordNetLemmatizer()
    
    # cleaning the text out of unnecessary characters
    clear_text = clean_text(expression, ' ', text)
    
    # make a token of the text
    token = tokenize(clear_text)
    
    # lemmatizing token
    lemm_text_list = [wnl.lemmatize(t) for t in token]
    
    return " ".join(lemm_text_list)

Лемматизируем каждый комментарий, и проведем его очистку от лишних символов:

In [13]:
%%time
data['lemm_text'] = data.text.apply(lambda x: lemmatize(x, expression=r'[^a-zA-z]'))

CPU times: user 1min 51s, sys: 586 ms, total: 1min 52s
Wall time: 1min 55s


In [14]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


Предварительная подготовка данных завершена.

Подгрузим и инициализируем английские стоп-слова. Стоп-слова не несут смысловой нагрузки, поэтому мы можем избавиться от них, тем самым облегчив нашу модель, не потеряв при этом в качестве.

In [15]:
stop_words = set(stopwords.words('english'))

Инициализируем векторайзер, который переведет наши комментарии в векторы:

In [16]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

Теперь, прежде чем переводить слова в векторное пространство, разобьем данные на тренировочную и тестовую выборку. Так как в противном случае получим лик в данных.

In [17]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

Проверим размерность выборок:

In [18]:
train_data.shape, test_data.shape

((143613, 3), (15958, 3))

Теперь векторизуем наш текст:

In [19]:
tf_idf_train = count_tf_idf.fit_transform(train_data.lemm_text)

In [20]:
tf_idf_test = count_tf_idf.transform(test_data.lemm_text)

Снова контролируем размер полученных выборок:

In [21]:
tf_idf_train.shape

(143613, 158354)

### Вывод:

- Данные проанализированы 
- Имеем сильный дисбаланс классов 9к1
- Данные разбиты на тренировочную и тестовую выборку
- Текст переведен в векторное прстранство

## Обучение

Учиться будем на простой модели классификации логистической регрессии. В качестве параметра передадим class_weight='balanced', чтобы сбалансировать классы. Инициализируем модель:

In [22]:
model = LogisticRegression(class_weight='balanced', random_state=12345)

Обучимся:

In [23]:
model.fit(tf_idf_train, train_data.toxic)

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=12345)

Оценим качество модели:

In [24]:
predictions = model.predict(tf_idf_test)

In [25]:
score = f1_score(test_data.toxic, predictions)
score

0.7512437810945273

### Вывод:

Досигнуть требуемого порога качества удалось. **0.751 < 0.75**. 

Возможно еще улучшить модель:
- Очистка комментариев от лишних символом не идеальна. Встречается **много строк с лишними символами** (напр. "_"). Качество может быть улучшено путем более тщательной обработки данных

## Модель обучения с BERT

<div class="alert alert-block alert-info">Код для BERT реализовал, но обучение происходит слишком медленно. Оставил максимально небольшое количество объектов, чтобы модель хотя бы просто отработала. </div>

In [26]:
tokenizer = transformers.BertTokenizer('bert_model_files/vocab.txt')

Отберем сбалансированную выборку состоящую из равного количества объектов каждого из классов:

In [27]:
trunc_data = pd.concat([data.loc[data['toxic'] == 0].sample(100, random_state=42),
                        data.loc[data['toxic'] == 1].sample(100, random_state=42)])

In [28]:
trunc_data.shape

(200, 3)

Перемешаем выборку:

In [29]:
trunc_data = trunc_data.sample(200, random_state=42)

Токенизируем данные

In [30]:
tokenized = trunc_data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [31]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [32]:
config = transformers.BertConfig.from_json_file(
    'bert_model_files/bert_config.json')
model = transformers.BertForPreTraining.from_pretrained(
    'bert_model_files/bert_model.ckpt', config=config, from_tf=True)

Создадим эмбединги

In [33]:
%%time
from tqdm import notebook
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(
        attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:, 0, :].numpy())


CPU times: user 6min 44s, sys: 1min 33s, total: 8min 18s
Wall time: 7min 49s


In [42]:
features = np.concatenate(embeddings)

In [43]:
len(features)

200

Делим выборку на тренировочную и тестовую:

In [44]:
train_features = features[:int(len(features)/2)]
test_features = features[int(len(features)/2):]

train_target = trunc_data['toxic'].iloc[:int(len(features)/2)]
test_target = trunc_data['toxic'].iloc[int(len(features)/2):]

Обучаем модель логистической регрессии:

In [45]:
logreg = LogisticRegression()
logreg.fit(train_features, train_target)

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [46]:
predictions = logreg.predict(test_features)

In [47]:
score = f1_score(test_target, predictions)
score

0.6813186813186813

In [48]:
dummy_pred = np.array([rnd.randint(0, 1) for i in range(len(test_target))])

In [49]:
dummy_score = f1_score(test_target, dummy_pred)
score

0.6813186813186813

## Выводы

На небольшой тренировочной выборке, в 0.001% от исходного датафрейма, модель логистической регрессии с BERT показывает результат такой же, как и случайная dummy-модель. 

Вычислительная сложность такой связки крайне высока.

Требуемой точности мы смогли достигнуть без использования BERT. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны